<a href="https://colab.research.google.com/github/isaacpy-cloud/TraderWolfs/blob/main/Traders.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install python-binance
!pip install catboost
!pip install matplotlib --upgrade
!pip install mpl_finance
!pip install tradingview-ta

     |████████████████████████████████| 184kB 7.2MB/s 
     |████████████████████████████████| 61kB 6.7MB/s 
     |████████████████████████████████| 286kB 10.2MB/s 
     |████████████████████████████████| 3.1MB 14.5MB/s 
     |████████████████████████████████| 3.2MB 27.4MB/s 
     |████████████████████████████████| 501kB 32.9MB/s 
     |████████████████████████████████| 81kB 7.7MB/s 
     |████████████████████████████████| 245kB 37.6MB/s 
     |████████████████████████████████| 65.7MB 114kB/s 
     |████████████████████████████████| 11.6MB 380kB/s 
ERROR: albumentations 0.1.12 has requirement imgaug<0.2.7,>=0.2.5, but you'll have imgaug 0.2.9 which is incompatible.
  Found existing installation: matplotlib 3.2.2
    Uninstalling matplotlib-3.2.2:
      Successfully uninstalled matplotlib-3.2.2


In [ ]:
import pandas as pd
import numpy as np
import time
import dateparser
import pytz
import json
import datetime as dt
from datetime import datetime, timedelta
from threading import Thread
import matplotlib.pyplot as plt
from mpl_finance import candlestick_ochl
import matplotlib.dates as mdates
from matplotlib.pyplot import figure
from sklearn import preprocessing, model_selection, neighbors, svm
from sklearn.preprocessing import StandardScaler, RobustScaler, MaxAbsScaler
from sklearn.metrics import accuracy_score, mean_squared_error, mean_absolute_error
from catboost import CatBoost, CatBoostClassifier, CatBoostRegressor, Pool, cv
from tqdm import tqdm as tqdm
from binance.client import Client
from binance.enums import *
from binance.websockets import BinanceSocketManager
from time import sleep
from tradingview_ta import TA_Handler, Interval, Exchange
from tradingview_ta import Compute

In [ ]:
def binance_data(klines):
  df = pd.DataFrame(klines.reshape(-1,12), dtype=float, columns=['Open Time','Open','High','Low','Close','Volume','Close Time','Quote Asset Volume','Number of Trades','Taker Buy Base Asset Volume','Taker buy quote asset volume','Can be ignored'])
  df['Open Time'] = pd.to_datetime(df['Open Time'], unit='ms')
  df['Close Time'] = pd.to_datetime(df['Close Time'], unit='ms')
  return df

In [ ]:
def total_analysis(symbol,interval):
  handler = TA_Handler(
    symbol=symbol,
    screener="crypto",
    exchange="binance",
    interval=interval
  )
  analysis = handler.get_analysis()
  return_data = analysis.summary
  if return_data['RECOMMENDATION'] == "BUY" or return_data['RECOMMENDATION'] == "STRONG_BUY":
    return True
  else:
    return False

In [ ]:
def total_oscillators(symbol,interval):
  handler = TA_Handler(
    symbol=symbol,
    screener="crypto",
    exchange="binance",
    interval=interval
  )
  analysis = handler.get_analysis()
  return_data = analysis.oscillators
  if return_data['RECOMMENDATION'] == "BUY" or return_data['RECOMMENDATION'] == "STRONG_BUY":
    return 1
  else:
    return 2

In [ ]:
def total_moving_averages(symbol,interval):
  handler = TA_Handler(
    symbol=symbol,
    screener="crypto",
    exchange="binance",
    interval=interval
  )
  analysis = handler.get_analysis()
  return_data = analysis.moving_averages
  if return_data['RECOMMENDATION'] == "BUY" or return_data['RECOMMENDATION'] == "STRONG_BUY":
    return 1
  else:
    return 2

In [ ]:
def parabolic_sar_signals(symbol,interval):
  handler = TA_Handler(
    symbol=symbol,
    screener="crypto",
    exchange="binance",
    interval=interval
  )
  analysis = handler.get_analysis()
  return_data = parabolic_sar_signals = Compute.PSAR(analysis.indicators["P.SAR"], analysis.indicators["open"])
  if return_data['RECOMMENDATION'] == "BUY" or return_data['RECOMMENDATION'] == "STRONG_BUY":
    return 1
  else:
    return 2

In [ ]:
def analysis_indicators_rsi(symbol,interval):
  handler = TA_Handler(
    symbol=symbol,
    screener="crypto",
    exchange="binance",
    interval=interval
  )
  analysis = handler.get_analysis()
  rsi_value = analysis.indicators['RSI']
  return rsi_value

In [ ]:
def analysis_indicators_macd_strategy(symbol,interval):
  handler = TA_Handler(
    symbol=symbol,
    screener="crypto",
    exchange="binance",
    interval=interval
  )
  analysis = handler.get_analysis()
  macd_signal = analysis.indicators['MACD.signal']
  return macd_signal

In [ ]:
def analysis_indicators_stoch_rsi(symbol,interval):
  handler = TA_Handler(
    symbol=symbol,
    screener="crypto",
    exchange="binance",
    interval=interval
  )
  analysis = handler.get_analysis()
  rsi_value = analysis.indicators['Stoch.K']
  return rsi_value

In [ ]:
def rsi_order_control(rsi_value):
  if rsi_value > 90:
    return True
  else:
    return False    

In [ ]:
def stoch_rsi_order_control(stoch_rsi_value):
  if stoch_rsi_value >= 95:
    return True
  else:
    return False

In [ ]:
def on_close(ws):
    print("Gizli Bağlantınız sonlandırılmıştır. Bot Trade İşlemlerini Durdurdu...")

In [ ]:
def order(side, quantity, symbol, order_type=ORDER_TYPE_MARKET):
    try:
        print("Ödeme Sağlanıyor...")
        order = client.create_order(symbol=symbol, side=side, type=order_type, quantity=quantity)
        print("Yapılan Ödeme İşlemi : " + order)
    except Exception as e:
        print("Bir sorun oluştu. Hata Kodu:  - {}".format(e))
        return False
    return True

def on_open(ws):
    ip_adress = get("https://api.ipify.org").text
    local_ip = socket.gethostbyname(socket.gethostname())
    get_adress = format(ip_adress)
    public_password = hasher.sha256()
    public_password.update(get_adress.encode('utf-8'))
    hasher_pass = public_password.hexdigest()
    hasher_newpass = '3LywfTjcbJYvdPN4vAwCBVagYa5nfnpzBb'
    print("Web Soketlerine Bağlandı. Gizli Bağlantı Kodunuz : " + hasher_pass)

In [ ]:
BOT_SYMBOL = "BTCTRY"
SOCKET_SYMBOL = BOT_SYMBOL.lower()

In [ ]:
def on_message(ws, message):
  print("Message")       

In [ ]:
ws = websocket.WebSocketApp(SOCKET, on_open=on_open, on_close=on_close, on_message=on_message)
ws.run_forever()